In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dataloader
from tqdm import tqdm

from model_architecture import Network

In [2]:
PROPOSAL_SIZE = (128, 128)
BATCH_SIZE = 100
BALANCE = 0.5

dataset_train = dataloader.PotholeDataset('./Potholes/annotated-images/', './Potholes/proposals/', './Potholes/annotated-images/', proposals_per_batch=BATCH_SIZE, proposal_size=PROPOSAL_SIZE, balance=BALANCE, split='train')
dataset_val = dataloader.PotholeDataset('./Potholes/annotated-images/', './Potholes/proposals/', './Potholes/annotated-images/', proposals_per_batch=BATCH_SIZE, proposal_size=PROPOSAL_SIZE, balance=BALANCE, split='val')
dataset_test = dataloader.PotholeDataset('./Potholes/annotated-images/', './Potholes/proposals/', './Potholes/annotated-images/', proposals_per_batch=BATCH_SIZE, proposal_size=PROPOSAL_SIZE, balance=BALANCE, split='test')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)

model = Network(proposal_size=PROPOSAL_SIZE)
model.apply(init_weights)
model.to(device)
#Initialize the optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.1)

cuda


In [4]:
def train(model, optimizer, num_epochs=10):
    def loss_fun(output, target):
        return F.cross_entropy(output, target)
    
    out_dict = {
              'train_acc': [],
              'val_acc': [],
              'train_loss': [],
              'val_loss': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        train_correct = 0
        train_loss = []
        # for minibatch_no, (data, target) in tqdm(enumerate(dataset), total=len(dataset)):
        for idx, (single_image_dict) in tqdm(enumerate(dataset_train), total=len(dataset_train)):
            # for proposal, label, proposal_image in zip(single_image_dict['proposals'], single_image_dict['labels'], single_image_dict['proposal_images']):
            proposal_image, label = single_image_dict['proposal_images'].to(device), single_image_dict['labels'].to(device)
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(proposal_image)
            #Compute the loss

            # print(f"Proposal shape: {single_image_dict['proposals'].shape if single_image_dict['proposals'] is not None else 'None'}")
            # print(f"Proposal image shape: {proposal_image.shape if proposal_image is not None else 'None'}")
            # print(f"Label: {label}")
            # print(f"Label shape: {label.shape if isinstance(label, torch.Tensor) else 'Not a Tensor'}")

            loss = loss_fun(output, label)
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (label==predicted).sum().cpu().item()

        #Comput the test accuracy
        val_loss = []
        val_correct = 0
        model.eval()
        for single_val_dict in dataset_val:
            # for proposal_val, label_val, proposal_image_val in zip(single_val_dict['proposals'], single_val_dict['labels'], single_val_dict['proposal_images']):
            proposal_image_val, label_val = single_val_dict['proposal_images'].to(device), single_val_dict['labels'].to(device)
            with torch.no_grad():
                output = model(proposal_image_val)

            val_loss.append(loss_fun(output, label_val).cpu().item())
            predicted = output.argmax(1)
            val_correct += (label_val==predicted).sum().cpu().item()

        out_dict['train_acc'].append(train_correct/len(dataset_train)/BATCH_SIZE)
        out_dict['val_acc'].append(val_correct/len(dataset_val)/BATCH_SIZE)
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['val_loss'].append(np.mean(val_loss))

        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(val_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['val_acc'][-1]*100:.1f}%") # Dividing by 5 because of the batch_size
        
    return out_dict

In [5]:
train(model, optimizer)

  0%|          | 0/10 [00:00<?, ?epoch/s]

c:\Users\samer\Documents\Intro to deep learning for computer vision\pothole_detector\dataloader.py:137: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels[selected_indices])
  0%|          | 0/10 [01:02<?, ?epoch/s]


KeyboardInterrupt: 